# Imports

In [ ]:
import sys
from pathlib import Path

ROOT = Path.cwd().parent
sys.path.append(str(ROOT))

import numpy as np
import torch
import torchvision.transforms as T
import matplotlib.pyplot as plt
from tqdm import tqdm

import faiss
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import STL10
from torchvision import models
import os 
import json

from src.config import load_config
from src.metrics import (
    compute_overlap_stats,
    compute_distance_divergence_stats,
    compute_lid_stats,
    compute_barycenter_stats,
)


from copy import deepcopy
from datetime import datetime

import yaml


# Load Configs

In [ ]:
cfg = load_config(
    "../config/base.yaml",
    "../config/data.yaml",
    "../config/embedding.yaml",
    "../config/search.yaml",
    "../config/ann.yaml",
)

In [ ]:
NUM_EXPERIMENTS = cfg["experiment"]["num_experiments"]
BASE_SEED = cfg["experiment"]["seed"]
SAMPLE_SIZE = cfg["data"]["sample"]["size"]

M = cfg["ann"]["hnsw"]["m"]
EF_CONSTRUCTION = cfg["ann"]["hnsw"]["ef_construction"]
EF_SEARCH = cfg["ann"]["hnsw"]["ef_search"]

K_MIN = cfg["search"]["knn"]["k_min"]
K_MAX = cfg["search"]["knn"]["k_max"]
K_STEP = cfg["search"]["knn"]["k_step"]
K_VALUES = list(range(K_MIN, K_MAX + 1, K_STEP))

RUNS_DIR = ROOT / Path(cfg["paths"]["runs_dir"])
RUNS_DIR.mkdir(exist_ok=True)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DATA_PATH = ROOT / cfg["data"]["data_path"]


p = cfg["embedding"]["preprocessing"]

In [ ]:
transform = T.Compose([
    T.Resize(p["resize"]),
    T.CenterCrop(p["center_crop"]),
    T.ToTensor(),
    T.Normalize(
        mean=p["normalize"]["mean"],
        std=p["normalize"]["std"]
    )
])

# Download The Dataset

In [ ]:
dataset = STL10(
    root=DATA_PATH,
    split="unlabeled",
    download=True,
    transform=transform
)

FULL_DATASET_LEN = len(dataset)


print("Full STL-10 size:", FULL_DATASET_LEN)


# Load the embedding model and remove the classification head. 

In [ ]:
# Load pretrained ResNet-50 (ImageNet-trained weights)
resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)

# Remove the final classification layer to expose penultimate embeddings
embedding_model = torch.nn.Sequential(*list(resnet.children())[:-1])
embedding_model.to(DEVICE)
embedding_model.eval()

# Embedding extraction function
def extract_embeddings(batch):
    with torch.no_grad():
        feats = embedding_model(batch.to(DEVICE))
        # Output shape: (batch_size, 2048, 1, 1) → squeeze to (batch_size, 2048)
        feats = feats.squeeze(-1).squeeze(-1)
    return feats.cpu().numpy().astype("float32")

print("Embedding dimension:", 2048)

In [ ]:
import numpy as np

MIN_RUNS = 5
WINDOW = 3                 # require stability for last 3 updates
TOL_ABS = 1e-3             # absolute tolerance for curve change (tune)
curve_history = []         # store per-run curves
delta_history = []         # store mean-curve change per run

def curve_delta(mean_prev, mean_curr, mode="max"):
    diff = np.abs(mean_curr - mean_prev)
    if mode == "max":
        return float(diff.max())
    elif mode == "rms":
        return float(np.sqrt(np.mean(diff**2)))
    else:
        raise ValueError("mode must be 'max' or 'rms'")



MAX_NUM_EXPERIMENTS = 100

for i in range(MAX_NUM_EXPERIMENTS):
    print(f"\n================ RUN {i} ================")

    # -----------------------------
    # Run setup
    # -----------------------------
    run_cfg = deepcopy(cfg)
    run_cfg["experiment"]["run_id"] = i
    
    
    run_cfg["data"]["sample"]["seed"] = BASE_SEED + i # BASE SEED + i to get new sample each iter

    run_id = f"{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}_run{i:02d}"
    run_dir = ROOT / RUNS_DIR / run_id
    run_dir.mkdir(exist_ok=False)

    with open(run_dir / "config.yaml", "w") as f:
        yaml.safe_dump(run_cfg, f)

    # -----------------------------
    # Dataset subsample (per run)
    # -----------------------------
    rng = np.random.default_rng(run_cfg["data"]["sample"]["seed"])
    
    # generates array of ints of size SAMPLE_SIZE within LEN of FULL_DATASET_LEN, 
    # without using same image mroe than 1 time.
    indices = rng.choice(FULL_DATASET_LEN, SAMPLE_SIZE, replace=False)
    
    # collects a sample as a "SUBSET" from full dataset according to the random indices
    # SUbset is an extension of DataLoader I think
    dataset_subset = Subset(dataset, indices)

    loader = DataLoader(
        dataset_subset,
        # TODO: update this from configs
        batch_size=run_cfg["embedding"]["batch_size"], 
        shuffle=False,
    )




    # -----------------------------
    # Embeddings (per run)
    # -----------------------------
    emb_file = run_dir / "embeddings.npy"
    emb_chunk_dir = run_dir / "emb_chunks"

    if emb_file.exists():
        embeddings = np.load(emb_file)
    else:
        emb_chunk_dir.mkdir(exist_ok=True)
        chunks = []

        for j, (batch, _) in enumerate(tqdm(loader)):
            emb = extract_embeddings(batch) # (batch, 2048)
            chunk_path = emb_chunk_dir / f"chunk_{j:05d}.npy"
            np.save(chunk_path, emb)
            chunks.append(chunk_path)

        embeddings = np.vstack([np.load(p) for p in chunks])
        np.save(emb_file, embeddings)

    #  collect dimension of the embeddings 
    d = embeddings.shape[1]
    print("Embeddings shape:", embeddings.shape)




    # -----------------------------
    # FAISS indices (per run)
    # -----------------------------

    # LINEAR SEARCH INDEX: 
    # Create a brute-force exact index that ranks 
    # neighbors by Euclidean distance.
    index_exact = faiss.IndexFlatL2(d)

    # Add embeddings to the index
    index_exact.add(embeddings)



    # HNSW ANN SEARCH INDEX:     
    # Build an HNSW graph index that ranks 
    # neighbors using Euclidean distance
    index_ann = faiss.IndexHNSWFlat(d, M, faiss.METRIC_L2)

    # set graph construction hyperparameter (larger = higher quality, slower build)
    index_ann.hnsw.efConstruction = EF_CONSTRUCTION

    # set search-time accuracy hyperparameter (larger = higher recall, slower search)
    index_ann.hnsw.efSearch = EF_SEARCH

    # add all embedding vectors to the ANN index
    index_ann.add(embeddings)
    
    # -----------------------------
    # Exact neighbors (per run)
    # -----------------------------
    exact_I_path = run_dir / "exact_I.npy"
    exact_D_path = run_dir / "exact_D.npy"

    if exact_I_path.exists() and exact_D_path.exists():
        I_exact_full = np.load(exact_I_path)
        D_exact_full = np.load(exact_D_path)
    else:
        D_exact_full, I_exact_full = index_exact.search(
            embeddings, len(embeddings)
        )
        np.save(exact_I_path, I_exact_full)
        np.save(exact_D_path, D_exact_full)




    # -----------------------------
    # k-sweep + metrics (per run)
    # -----------------------------
    results = {
        "k": [],
        "mean_overlap": [],
        "std_overlap": [],
        "mean_dist_divergence": [],
        "std_dist_divergence": [],
        "mean_barycenter_shift": [],
        "std_barycenter_shift": [],
        "mean_lid_diff": [],
        "std_lid_diff": [],
        "lid_exact": [],
        "lid_ann": [],
    }


    for k in K_VALUES:
        
        # We ask for k+1 and drop the first value to prevent 
        # including the distance = 0 of a self returned query point. 
        # ---- Exact: slice and drop self-match ----
        D_exact = D_exact_full[:, 1:k+1]
        I_exact = I_exact_full[:, 1:k+1]

        # ---- ANN: search k+1 and drop self-match ----
        D_ann, I_ann = index_ann.search(embeddings, k + 1)
        D_ann = D_ann[:, 1:]
        I_ann = I_ann[:, 1:]

        mean_ov, std_ov = compute_overlap_stats(I_exact, I_ann, k)
    
        mean_dist_divergence, std_dist_divergence = compute_distance_divergence_stats(D_exact, D_ann)
        
        bary_shift, std_bary_shift = compute_barycenter_stats(embeddings, I_exact, I_ann, D_exact)
                
        lid_stats = compute_lid_stats(D_exact, D_ann)

        

        results["k"].append(k)
        
        results["mean_overlap"].append(mean_ov)
        results["std_overlap"].append(std_ov)
        
        results["mean_dist_divergence"].append(mean_dist_divergence)
        results["std_dist_divergence"].append(std_dist_divergence)
        
        results["mean_barycenter_shift"].append(bary_shift)
        results["std_barycenter_shift"].append(std_bary_shift)
        
        results["mean_lid_diff"].append(lid_stats["mean_lid_diff"])
        results["std_lid_diff"].append(lid_stats["std_lid_diff"])
        results["lid_exact"].append(lid_stats["mean_lid_exact"])
        results["lid_ann"].append(lid_stats["mean_lid_ann"])



    # -----------------------------
    # Save results + plots (per run)
    # -----------------------------
    with open(run_dir / "metrics.json", "w") as f:
        json.dump(results, f, indent=2)

    plots_dir = run_dir / "plots"
    plots_dir.mkdir(exist_ok=True)

    # Overlap vs k
    plt.figure()
    plt.plot(results["k"], results["mean_overlap"])
    plt.xlabel("k")
    plt.ylabel("Mean Overlap")
    plt.title("Overlap vs k")
    plt.savefig(plots_dir / "overlap_vs_k.png")
    plt.close()

    # LID difference vs k
    plt.figure()
    plt.plot(results["k"], results["mean_lid_diff"])
    plt.xlabel("k")
    plt.ylabel("Mean LID Difference (ANN − Exact)")
    plt.title("LID Difference vs k")
    plt.savefig(plots_dir / "lid_vs_k.png")
    plt.close()

    # Barycenter shift vs k
    plt.figure()
    plt.plot(results["k"], results["mean_barycenter_shift"])
    plt.xlabel("k")
    plt.ylabel("Mean Normalized Barycenter Shift")
    plt.title("Barycenter Shift vs k")
    plt.savefig(plots_dir / "barycenter_shift_vs_k.png")
    plt.close()


    # Store this run's curve
    curve = np.array(results["mean_dist_divergence"], dtype=float)

    if not np.isfinite(curve).all():
        print("Warning: non-finite divergence values detected, skipping convergence check.")
    else:
        curve_history.append(curve)


    # Once we have at least 2 runs, track mean-curve change
    if len(curve_history) >= 2:
        mean_prev = np.mean(curve_history[:-1], axis=0)
        mean_curr = np.mean(curve_history, axis=0)

        delta = curve_delta(mean_prev, mean_curr, mode="max")  # or "rms"
        delta_history.append(delta)

        print(f"Mean distance-divergence curve change Δ = {delta:.6g}")

    # Stop rule: stable for WINDOW consecutive updates (after MIN_RUNS)
    if len(curve_history) >= MIN_RUNS and len(delta_history) >= WINDOW:
        if all(d < TOL_ABS for d in delta_history[-WINDOW:]):
            print(f"\nDistance-divergence curve stabilized after {len(curve_history)} runs.")
            break

